
# Packing CAT Cell demo notebook
The main functions of this demo are located in [chordal_axis_transform.py](../irregular_object_packing/packing/chordal_axis_transform.py)


### Contents
1. Simple CAT Cell demo
2. Larger Demo with cylindrical container and multiple blood cell meshes.

### Pre-requisites
Please install the required packages using the following command:

```bash
pip install -r requirements.txt
```




## Basic CAT Cell demo
The cell below shows a simple demo of computing cat faces. The container is a cube and the object os a cuboid rotated 45 degrees over the z axis. The red faces are the CAT faces that belong to the cube in the back. The yellow arrows are the normal vectors, which point inwards, of each face. the yellow faces are the cat faces that belong to the other objects.

## 4 Cubes

In [1]:
from irregular_object_packing.cat.chordal_axis_transform import compute_cdt, compute_cat_faces, filter_relevant_cells
import numpy as np
import pyvista as pv

from irregular_object_packing.cat.utils import get_tetmesh_cell_arrays

container = pv.Cube(center=(0, 0, 0), x_length=4, y_length=4, z_length=3).triangulate().clean()

cube4 = pv.Cube(center=(-1, -1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube1 = pv.Cube(center=(1, 1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube2 = pv.Cube(center=(1, -1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube3 = pv.Cube(center=(-1, 1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube5 = pv.Cube(center=(0, 0, 0), x_length=0.9, y_length=2, z_length=1).triangulate().clean()

meshes = [cube1, cube2, cube3, cube4]#, cube5]
tetmesh = compute_cdt(meshes + [container])
boxes_mesh = pv.PolyData()
for mesh in meshes: boxes_mesh.merge(mesh, merge_points=False, inplace=True)

# plotter = pv.Plotter()
# plotter.add_mesh_clip_plane(tetmesh.explode(), color="white", opacity=1, show_edges=True, crinkle=True)
# plotter.show()

tet_cells = get_tetmesh_cell_arrays(tetmesh)
npoints_for_meshes = [mesh.n_points for mesh in meshes]
npoints_for_meshes.append(container.n_points)
relevant_cells, filtered_out = filter_relevant_cells(tet_cells, npoints_for_meshes)
assert len(filtered_out) != 0
# relevant_cells = tetmesh.extract_cells([cell.id for cell in relevant_cells])

plotter = pv.Plotter()
plotter.add_mesh(boxes_mesh, color="red", opacity=0.5, show_edges=True)
plotter.add_points(tetmesh.points, color="black", opacity=1, show_vertices=True)
# plotter.add_mesh_clip_plane(tet_points, color="white", opacity=1, show_edges=True)
# plotter.add_mesh_clip_plane(relevant_cells, color="white", opacity=1, show_edges=True, crinkle=False)
plotter.show()



Delaunizing vertices...
Delaunay seconds:  0.000191

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  7e-06
Total running seconds:  0.00022

Statistics:

  Input points: 40

  Mesh points: 40
  Mesh tetrahedra: 154
  Mesh faces: 314
  Mesh edges: 199
  Convex hull faces: 12



Widget(value="<iframe src='http://localhost:60890/index.html?ui=P_0x1323d5ba0_0&reconnect=auto' style='width: …

In [6]:

from irregular_object_packing.mesh.utils import convert_faces_to_polydata_input


n_points_per_obj = [len(mesh.points) for mesh in meshes] + [len(container.points)]
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])




cat_meshes = []
for cat_cell in cat_cells[:-1]:
    cat_meshes.append(pv.PolyData(*convert_faces_to_polydata_input(cat_cell)))
    
def print_cat_analysis(cat_meshes):
    for i, cat in enumerate(cat_meshes):
        print(f"cat {i}: {cat.n_points}, is manifold: {cat.is_manifold}")


print_cat_analysis(cat_meshes)
plotter = pv.Plotter()
# plotter.add_mesh_clip_plane(tetmesh, color="grey", show_edges=True, opacity=0.9, crinkle=True)
for mesh in meshes:
    plotter.add_mesh(mesh, color="red", show_edges=True, opacity=1,)

plotter.show()



cat 0: 78, is manifold: True
cat 1: 58, is manifold: True
cat 2: 58, is manifold: True
cat 3: 62, is manifold: True


Widget(value="<iframe src='http://localhost:60890/index.html?ui=P_0x10d17a410_4&reconnect=auto' style='width: …

In [7]:
from irregular_object_packing.mesh.collision import compute_collision


cell_id = 3
def plot_single_mesh_with_cat(cell_id):
    plotter = pv.Plotter()
    plotter.add_mesh(meshes[cell_id], color="red", show_edges=True, opacity=1,)
    plotter.add_mesh(cat_meshes[cell_id].explode(), color="yellow", show_edges=True, opacity=0.8,)
    plotter.show()

plot_single_mesh_with_cat(cell_id)

Widget(value="<iframe src='http://localhost:60890/index.html?ui=P_0x156804340_5&reconnect=auto' style='width: …

## Sphere inside a sphere

In [8]:

container = pv.Sphere(radius=2).triangulate().clean().rotate_y(1)
sphere = pv.Sphere(radius=1).triangulate().clean()
CDT_DEFAULTS = {
    "nobisect": True,
    "steinerleft": 0,
    "minratio": 10.0,
    "quality": False,
    "cdt": True,
    "switches": "O0/0",
}
meshes = [sphere, container]#, cube5]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)
# assert(len(tetmesh.points) == len(sphere.points) + len(container.points))
# assert(len(tetmesh.points) == tetmesh.n_points)

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]

# TODO: Filter out steinerpoints. Probably the last points.
print(f'number of steiner points: {len(steiner_points)}')

Delaunizing vertices...
number of steiner points: 0
Delaunay seconds:  0.004352

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  8.6e-05
Total running seconds:  0.004484

Statistics:

  Input points: 1684

  Mesh points: 1684
  Mesh tetrahedra: 8916
  Mesh faces: 18672
  Mesh edges: 11439
  Convex hull faces: 1680



In [9]:
n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
# FIXME: There are only 2 objects in this situation, however, there are object ids 0,1 and 2
print_cat_analysis(cat_meshes)

cat 0: 3478, is manifold: True
cat 1: 3478, is manifold: True


In [10]:
plotter = pv.Plotter()
plotter.add_mesh(sphere, color="red", show_edges=True, opacity=0.8)
plotter.add_mesh(container, color="white", opacity=0.8)
plotter.add_mesh(cat_meshes[0], color="yellow", show_edges=True, opacity=0.8)
if len(steiner_points) != 0:
    plotter.add_points(steiner_points, color="purple", render_points_as_spheres=True, point_size=10)
plotter.show()


Widget(value="<iframe src='http://localhost:60890/index.html?ui=P_0x156928a90_6&reconnect=auto' style='width: …

In [11]:
container = pv.Sphere(radius=2).triangulate().clean().rotate_y(1)
sphere_left = pv.Sphere(radius=0.5, center=(-0.6,0,0)).triangulate().clean()
sphere_right = pv.Sphere(radius=0.5, center=(0.6,0,0)).triangulate().clean()
meshes = [sphere_left, sphere_right, container]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)
tetmesh = tetmesh.clean()

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
print("steiner points: ", len(steiner_points))

n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
print_cat_analysis(cat_meshes)


Delaunizing vertices...
steiner points:  0
Delaunay seconds:  0.008941

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  0.000135
Total running seconds:  0.009141

Statistics:

  Input points: 2526

  Mesh points: 2526
  Mesh tetrahedra: 14808
  Mesh faces: 30456
  Mesh edges: 18173
  Convex hull faces: 1680

cat 0: 3443, is manifold: True
cat 1: 3443, is manifold: True
cat 2: 5110, is manifold: True


In [12]:
plotter= pv.Plotter()
def plot_cat_cells(plotter):
    for mesh in meshes[:-1]:
        plotter.add_mesh(mesh, color="red", show_edges=True, opacity=0.8)
    for mesh in cat_meshes[:-1]:
        plotter.add_mesh(mesh, color="yellow", show_edges=True, opacity=0.8)

    plotter.add_mesh(container, color="white", opacity=0.8)
    plotter.add_mesh(cat_meshes[-1], color="green", show_edges=True, opacity=0.8)

    if len(steiner_points) != 0:
        plotter.add_points(steiner_points, color="purple", render_points_as_spheres=True, point_size=20)
    plotter.show()

plot_cat_cells(plotter)

plotter= pv.Plotter()
plotter.add_mesh(container, color="white", opacity=0.8)
plotter.add_mesh(cat_meshes[-1], color="green", show_edges=True, opacity=0.8)
plotter.show()

Widget(value="<iframe src='http://localhost:60890/index.html?ui=P_0x15683d420_7&reconnect=auto' style='width: …

Widget(value="<iframe src='http://localhost:60890/index.html?ui=P_0x15683d480_8&reconnect=auto' style='width: …

In [13]:
container = pv.Sphere(radius=1, theta_resolution=10, phi_resolution=10).triangulate().clean().rotate_y(1)
sphere1 = pv.Sphere(radius=0.2, center=(-0.6,0,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere2 = pv.Sphere(radius=0.2, center=(0.6,0,0) , theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere3 = pv.Sphere(radius=0.2, center=(0,0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere4 = pv.Sphere(radius=0.2, center=(0,-0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()

meshes = [sphere1, sphere2, sphere3, sphere4, container]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)
tetmesh = tetmesh.clean()

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
print("steiner points: ", len(steiner_points))


n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
print_cat_analysis(cat_meshes)

Delaunizing vertices...
steiner points:  0
Delaunay seconds:  0.002015

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  2.4e-05
Total running seconds:  0.00207

Statistics:

  Input points: 410

  Mesh points: 410
  Mesh tetrahedra: 2334
  Mesh faces: 4748
  Mesh edges: 2823
  Convex hull faces: 160

cat 0: 411, is manifold: True
cat 1: 411, is manifold: True
cat 2: 394, is manifold: True
cat 3: 394, is manifold: True
cat 4: 842, is manifold: True


In [14]:
plot_cat_cells(pv.Plotter())

plotter = pv.Plotter()
plotter.add_mesh(cat_meshes[-1], color="red", show_edges=True, opacity=0.8)
plotter.show()



Widget(value="<iframe src='http://localhost:60890/index.html?ui=P_0x156876200_9&reconnect=auto' style='width: …

Widget(value="<iframe src='http://localhost:60890/index.html?ui=P_0x1568756f0_10&reconnect=auto' style='width:…

In [15]:
container = pv.Sphere(radius=1, theta_resolution=10, phi_resolution=10).triangulate().clean().rotate_y(1)
sphere1 = pv.Sphere(radius=0.2, center=(-0.6,0,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere2 = pv.Sphere(radius=0.2, center=(0.6,0,0) , theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere3 = pv.Sphere(radius=0.2, center=(0,0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere4 = pv.Sphere(radius=0.2, center=(0,-0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
cube_5 = pv.Cube(center=(0,0,0.6), x_length=0.2, y_length=0.2, z_length=0.2).triangulate().clean()
cube_6 = pv.Cube(center=(0,0,-0.6), x_length=0.2, y_length=0.2, z_length=0.2).triangulate().clean()


meshes = [sphere1, sphere2, sphere3, sphere4, cube_5, cube_6, container]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)
tetmesh = tetmesh.clean()

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
print("steiner points: ", len(steiner_points))

n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
print_cat_analysis(cat_meshes)

Delaunizing vertices...
steiner points:  0
Delaunay seconds:  0.002105

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  2.3e-05
Total running seconds:  0.002159

Statistics:

  Input points: 426

  Mesh points: 426
  Mesh tetrahedra: 2400
  Mesh faces: 4880
  Mesh edges: 2905
  Convex hull faces: 160

cat 0: 383, is manifold: True
cat 1: 162, is manifold: True
cat 2: 155, is manifold: True
cat 3: 613, is manifold: True
cat 4: 518, is manifold: True
cat 5: 735, is manifold: True
cat 6: 922, is manifold: True


In [16]:
plot_cat_cells(pv.Plotter())

plotter = pv.Plotter()
plotter.add_mesh(cat_meshes[-1], color="red", show_edges=True, opacity=0.8)
plotter.show()

Widget(value="<iframe src='http://localhost:60890/index.html?ui=P_0x29249bb20_11&reconnect=auto' style='width:…

Widget(value="<iframe src='http://localhost:60890/index.html?ui=P_0x292498910_12&reconnect=auto' style='width:…

In [17]:
from irregular_object_packing.mesh.sampling import resample_pyvista_mesh
from irregular_object_packing.mesh.transform import scale_to_volume


container = pv.Sphere(radius=1, theta_resolution=10, phi_resolution=10).triangulate().clean().rotate_y(1)
cell_mesh = pv.read("./data/RBC_normal.stl").triangulate().clean()
cell_mesh = resample_pyvista_mesh(cell_mesh, 100)
target_width = cell_mesh.volume**(1/3) *2 / container.volume**(1/3)
cell_mesh = scale_to_volume(cell_mesh,) # target like sphere
cell_mesh1 = cell_mesh.copy().translate((0.6,0,0))
cell_mesh2 = cell_mesh.copy().translate((-0.6,0,0))
cell_mesh3 = cell_mesh.copy().translate((0,0.6,0))
cell_mesh4 = cell_mesh.copy().translate((0,-0.6,0))
cell_mesh5 = cell_mesh.copy().translate((0,0,0.6))
cell_mesh6 = cell_mesh.copy().translate((0,0,-0.6))

sphere1 = pv.Sphere(radius=0.2, center=(-0.6,0,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere2 = pv.Sphere(radius=0.2, center=(0.6,0,0) , theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere3 = pv.Sphere(radius=0.2, center=(0,0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere4 = pv.Sphere(radius=0.2, center=(0,-0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
cube_5 = pv.Cube(center=(0,0,0.6), x_length=0.2, y_length=0.2, z_length=0.2).triangulate().clean()
cube_6 = pv.Cube(center=(0,0,-0.6), x_length=0.2, y_length=0.2, z_length=0.2).triangulate().clean()

meshes = [sphere1, sphere2, sphere3, sphere4, cube_5, cube_6, container]
meshes = [cell_mesh1, cell_mesh2, cell_mesh3, cell_mesh4, cell_mesh5, cell_mesh6, container]


# meshes = [sphere1, sphere2, sphere3, sphere4, cube_5, cube_6, container]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)
tetmesh = tetmesh.clean()

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
print("steiner points: ", len(steiner_points))

n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
print_cat_analysis(cat_meshes)

FileNotFoundError: File (/Users/maurits/code/cls/thesis/irregular-object-packing/notebooks/data/RBC_normal.stl) not found